In [1]:
import json
import config
import pandas as pd

In [4]:
# {paper_id:{id:'',tittle:'',abstract:'<长文本>',keywords:[],authors:[{name:'',org:''},venue:'',year:'']}} 
# authors: 中文名LI Heng
test_pub = json.load(open(config.test_pub_path, mode='r', encoding='utf-8'))

# {authors:[id,...]} hengli
test_author = json.load(open(config.test_author_path, mode='r', encoding='utf-8'))

true_pub = json.load(open(config.true_pub_path, mode='r', encoding='utf-8'))
true_author = json.load(open(config.true_author_path, mode='r', encoding='utf-8'))

train_pub = json.load(open(config.train_pub_path, mode='r', encoding='utf-8'))
train_author = json.load(open(config.train_author_path, mode='r', encoding='utf-8'))

# {authors:[[id,...],[],[]]}
test_cluster = None

In [5]:
org_name, authors_name = [], []

for i in test_pub:
    for j in test_pub[i]:
        for k in test_pub[i]['authors']:
            if "org" in k.keys():
                if k['org'] != "":
                    org_name.append(k['org'])
                else:
                    org_name.append('NO_ORG')
                authors_name.append(k['name'])
            else:
                org_name.append('NO_ORG')
                authors_name.append(k['name'])

org_authors_statics = pd.DataFrame({'org':org_name, 'authors':authors_name})

In [6]:
from bert_pre_train import BertPreTrain
pre_train_model = BertPreTrain(mode='pre_train', language='en', padding=False,
                                            embedding_dim=config.embedding_dim)

pre train feature loaded.


In [4]:
# 统计每个聚类里文章的数量
statistic_paper_num = []
for _name in train_author.keys():
    for _cluster_id in  train_author[_name]:
        statistic_paper_num.append(len(train_author[_name][_cluster_id]))
_ = pd.Series(statistic_paper_num)
pd.set_option('display.max_rows', 300)
print(_.value_counts())
print(len(_.value_counts()))

1       9199
2       2929
3       1879
4       1385
5       1004
6        731
7        618
8        534
9        460
11       344
10       336
12       296
13       268
14       222
15       198
16       168
17       152
18       129
19       113
20       113
21       103
22        96
25        78
24        74
23        74
27        65
28        64
26        58
30        45
29        45
31        42
36        41
32        40
33        39
34        37
35        35
37        30
38        25
40        25
41        23
42        23
39        20
48        18
43        16
44        15
45        15
46        15
56        15
49        14
55        14
47        14
53        13
61        12
62        11
51         9
72         9
57         9
58         9
60         9
54         9
52         8
64         8
66         8
98         8
65         8
83         8
78         7
97         7
50         7
69         7
77         7
81         7
76         6
71         6
82         5
63         5
92         5

In [5]:
import pickle
keyword_dict = pickle.load(open('../my_data/keyword_feature_map_dict.pkl', mode='rb'))
# with open('../my_data/keywod_embedding_可视化.txt', mode='w', encoding='utf-8') as fk:
#     for _keyword in keyword_dict.keys():
#         a = pre_train_model.get_output([_keyword], _show_tokens=False).astype('str').tolist()[0][0]
#         fk.writelines('{}\n'.format('\t'.join(a)))

In [6]:
keyword_dict

{'central': 0,
 0: 'central',
 'nervous': 1,
 1: 'nervous',
 'drugs': 2,
 2: 'drugs',
 'GC-FID': 3,
 3: 'GC-FID',
 'GC-MS': 4,
 4: 'GC-MS',
 'solid-phase': 5,
 5: 'solid-phase',
 'extraction': 6,
 6: 'extraction',
 'X-5': 7,
 7: 'X-5',
 'resin': 8,
 8: 'resin',
 'Branched': 9,
 9: 'Branched',
 'building': 10,
 10: 'building',
 'block': 11,
 11: 'block',
 'Cascade': 12,
 12: 'Cascade',
 'molecules': 13,
 13: 'molecules',
 'Initiator': 14,
 14: 'Initiator',
 'core': 15,
 15: 'core',
 'Three-directional': 16,
 16: 'Three-directional',
 'ether-bond': 17,
 17: 'ether-bond',
 'branched': 18,
 18: 'branched',
 'compounds': 19,
 19: 'compounds',
 'Ether': 20,
 20: 'Ether',
 'bond': 21,
 21: 'bond',
 'Four': 22,
 22: 'Four',
 '-': 23,
 23: '-',
 'and': 24,
 24: 'and',
 'six': 25,
 25: 'six',
 'directional': 26,
 26: 'directional',
 'ether': 27,
 27: 'ether',
 'Initial': 28,
 28: 'Initial',
 'Topology': 29,
 29: 'Topology',
 '': 30,
 30: '',
 'Cholecystokinin-4': 31,
 31: 'Cholecystokinin-4',
 '

In [7]:
org_authors_statics[org_authors_statics['authors'] == 'Hao Zhang']['org'].unique()

array(['NO_ORG', 'Department of Food Science',
       'State Key Laboratory of Networking and Switching Technology', ...,
       'National University of Singapore',
       'School of Computing, National University of Singapore, Singapore',
       'national university of singapore'], dtype=object)

In [10]:
# origin_data: 文章属性信息; content_data：文章内容信息
with open(config.origin_data, mode='w', encoding='utf-8') as f1:
    with open(config.content_data, mode='w', encoding='utf-8') as f2:
        for paper_id in train_pub:
            # paper_id, name_org, tittle, content, keywords, venue, year
            name_org = []
            for k in train_pub[paper_id]['authors']:
                if "org" in k.keys():
                    if k['org'] != "":
                        name_org.append(k['name'] + '@' + k['org'])
                    else:
                        name_org.append(k['name'] + '@' + 'NO_ORG')
                else:
                    name_org.append(k['name'] + '@' + 'NO_ORG')
            name_org = '|'.join(name_org)
            if isinstance(name_org, type(None)):
                name_org = ''

            tittle = train_pub[paper_id]['title']
            if isinstance(tittle, type(None)):
                tittle = ''

            if 'abstract' in train_pub[paper_id].keys():
                content = train_pub[paper_id]['abstract']
            else:
                content = ''
            if isinstance(content, type(None)):
                content = ''

            if 'keywords' in train_pub[paper_id].keys():
                keywords = train_pub[paper_id]['keywords']
                keywords = '|'.join(keywords)
            else:
                keywords = ''
            if isinstance(keywords, type(None)):
                keywords = ''

            venue = train_pub[paper_id]['venue']

            if 'year' in train_pub[paper_id].keys():
                year = train_pub[paper_id]['year']
            else:
                year = ''
            f1.writelines('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(paper_id,
                                                                name_org.replace('\r', '').replace('\n','').strip(),
                                                                tittle.replace('\r', '').replace('\n','').strip(),
                                                                content.replace('\r', '').replace('\n', '').strip(),
                                                                keywords.replace('\r', '').replace('\n', '').strip(), venue, year))
            f2.writelines('{}\n'.format(content.replace('\r', '').replace('\n', '').strip()))


In [5]:
# origin_test_data: 文章属性信息; content_test_data：文章内容信息
with open(config.true_origin_data, mode='w', encoding='utf-8') as f1:
    with open('../my_data/content_true.txt', mode='w', encoding='utf-8') as f2:
        for paper_id in true_pub:
            # paper_id, name_org, tittle, content, keywords, venue, year
            name_org = []
            for k in true_pub[paper_id]['authors']:
                if "org" in k.keys():
                    if k['org'] != "":
                        name_org.append(k['name'] + '@' + k['org'])
                    else:
                        name_org.append(k['name'] + '@' + 'NO_ORG')
                else:
                    name_org.append(k['name'] + '@' + 'NO_ORG')
            name_org = '|'.join(name_org)
            if isinstance(name_org, type(None)):
                name_org = ''

            tittle = true_pub[paper_id]['title']
            if isinstance(tittle, type(None)):
                tittle = ''

            if 'abstract' in true_pub[paper_id].keys():
                content = true_pub[paper_id]['abstract']
            else:
                content = ''
            if isinstance(content, type(None)):
                content = ''

            if 'keywords' in true_pub[paper_id].keys():
                keywords = true_pub[paper_id]['keywords']
                keywords = '|'.join(keywords)
            else:
                keywords = ''
            if isinstance(keywords, type(None)):
                keywords = ''

            venue = true_pub[paper_id]['venue']

            if 'year' in true_pub[paper_id].keys():
                year = true_pub[paper_id]['year']
            else:
                year = ''
            f1.writelines('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(paper_id,
                                                                name_org.replace('\r', '').replace('\n','').strip(),
                                                                tittle.replace('\r', '').replace('\n','').strip(),
                                                                content.replace('\r', '').replace('\n', '').strip(),
                                                                keywords.replace('\r', '').replace('\n', '').strip(), venue, year))
            f2.writelines('{}\n'.format(content.replace('\r', '').replace('\n', '').strip()))

In [8]:
print(train_pub['LAKfuSaP'])

{'id': 'LAKfuSaP', 'title': 'Guochun Zhao Gondwana Research 10 (2006) 400-401. doi:10.1016/j.gr.2006.05.004 journal Gondwana Research Copyright © 2006 International Association for Gondwana Research Published by Elsevier B.V. All rights reserved. Elsevier B.V. 1342-937X 10 3 2006 2006 400-401 400 401 10.1016/j.gr.2006.05.004 http://dx.doi.org/10.1016/j.gr.2006.05.004 doi:10.1016/j.gr.2006.05.004', 'abstract': '', 'keywords': [], 'authors': [{'name': 'Guochun Zhao', 'org': 'Department of Earth Sciences, The University of Hong Kong, Pokfulam Road, Hong Kong'}], 'venue': '', 'year': ''}


In [9]:
train_author[list(train_author.keys())[0]]

def merge_paper_list(_dict_a):
    # 合并训练集中，同一作者下的文章，返回文章列表
    out_list = []
    for i in _dict_a:
        out_list.extend(_dict_a[i])
    return out_list

In [10]:
import pickle

In [11]:
a = pickle.load(open('../my_data/author_feature_map.pkl', mode='rb'))

In [12]:
# 某作者下的文章
out_list = merge_paper_list(train_author[list(train_author.keys())[0]])
out_list_feature = dict()

import numpy as np

sim_matrix = np.zeros((len(out_list),len(out_list)))

with open('../my_data/author_feature_simple.txt', mode='r', encoding='utf-8') as fa:
    for line in fa.readlines():
        paper_id, feature = line.strip().split('\t')
        feature_set = set(feature.split('|'))
        
        if paper_id in out_list:
            out_list_feature[paper_id] = feature_set
    
    for id_a, paper_id_a in enumerate(out_list):
        for id_b, paper_id_b in enumerate(out_list):
            sim_matrix[id_a, id_b] = len(out_list_feature[paper_id_a] & out_list_feature[paper_id_b])

In [13]:
out_list = merge_paper_list(train_author[list(train_author.keys())[0]])
print(out_list)

['P9a1gcvg', 'Rg5fAeTd', 'lJPsGNBE', 'er5gTz90', 'UG32p2zs', 'FTweBQNS', 'MqQOkfGH', 'HcrsMIFk', 'O14akltW', 'gkO4nqtJ', 'pW2XD7jw', '9VmupIoc', 'pPjljRgW', 'rakLh2IJ', '7zYwBnXA', '0raOqO0t', 'ct2XgIla', 'Vh2Yy9vh', 'Yqpk8Jyt', 'xiuPYHlY', 'xWDIT1E6', 'bUEtpqgY', 'L98DkDap', 'UnhHr74J', 'CvJPSUxQ', 'AriXov6L', 'elqOgbWS', 'dikitupP', 'nlwdBqLy', 'pT5EU4aH', 'ub0fgak2', 'eThzWtj0', '3p1wV47k', '9iQCpuHx', 'QFAYRwHl', 'lioaGlmw', 'vW26N7Jw', 'Y8dEQrwa', 'AmLblp6l', 'gsL5mZJL', 'TXiFqOQn', 'dKTBlS35', 'fKt8D2xH', 'qaykvKo3', 'W0P2muEr', 'sLjQJNj8', '0ytlj6Jw', '3P3o3ZQ9', 'sqGfmgG1', '2EGB9ZLn', 'wd3ugFQM', 'pYKTrEQj', 'ZUfWc0Zj', 'jMErL46k', 'g2a7ae8r', 'C0LwTl95', '84iQz3xI', 'saq73SMR', '7609LUea', 'mO4rXXob', 'Fhz4Q5J5', 'lUuqOzwx', 'Q8KEEDfP', 'vKhZWMKD', 'V2XqV0Gp', 'ceGwETv1', 'aSqmClGz', 'DRx4shZr', '1yu8n8Sn', 'kMO2ex8C', 'tLWwSkZD', '8a1YPO1b', '2j7eBdaC', 'kmyawALz', 'KM70zskA', 'APIk8oID', 'GSwb2A2E', 'b6L5kmwf', 'xYpXgCwE', 'PT14nnmM', 'j0xsYN8v', 'Ef93mr5E', 'nNiMjv7s', 'vx

In [55]:
for idx, i in enumerate(list(sim_matrix[out_list.index('FTweBQNS')])):
    if i >= 2:
        print(out_list[idx])

UG32p2zs
FTweBQNS


In [17]:
def read_paper_embedding():
    with open('train_paper_embedding.txt', mode='r', encoding='utf-8') as f1:
        

(1464, 1464)

In [14]:
[len(train_author[list(train_author.keys())[i]]) for i in range(len(list(train_author.keys())))].index(588)
train_author[list(train_author.keys())[203]]

{'0296NTh7': ['iRlhVbCa'],
 '04AD4CgF': ['LjzlDE7N', 'nwNljvmj', 'KmQA2tjm', 'TsQaWaFx'],
 '05ndzYMS': ['0RhtHBOz'],
 '0KOreGp7': ['XRvWk6rB'],
 '0NhkDdJA': ['pufq1Avx', 'rIyh2hym'],
 '0SgOlhjw': ['AtyufaaE', 'YkyZgJU8'],
 '0VQSIwW4': ['3cTzUEXY'],
 '0hAhY57S': ['vfsUcnWX', 'tnvFH838', 'hqPQiIIG'],
 '0kToMQ4d': ['rvbF908n'],
 '0l04TxRg': ['YCRTFEue', 'xjObHFjX', 'mtekSjBB', 'GmokmMmi'],
 '11IMHClI': ['aweFOi8d'],
 '1Gv5OCmN': ['KmJOPGwy'],
 '1JKXfRmK': ['ddbzwOSX'],
 '1Ku6GApU': ['qPPbpOR5', 'sklkUBPc', '6gJRdLkn'],
 '1NRufrAc': ['yFvBtJfw'],
 '1WZNG8R1': ['suLyivKP'],
 '1g5kqsiD': ['iTY6WnGf', '8DzTMDoH'],
 '1hawmhoE': ['i3YFkNcj'],
 '1rr4M14o': ['x3bBgG5o'],
 '1wdDBozQ': ['dKZzNY0y'],
 '23zwzOsn': ['CWqb0mp9'],
 '26R7yn6l': ['dSzdqizv'],
 '27rASY53': ['yxycY5Ie', '1ngEjn9t'],
 '284vClXD': ['lXAgIR8Q', 'UvXzK6iU'],
 '2DoIcsr7': ['qp1wWruB'],
 '2FmIT6v9': ['eOTlOVSE'],
 '2JaAtbII': ['1h8jNEdv'],
 '2QjEs2y2': ['SA9RCAV6'],
 '2WgTz8zd': ['QdqcEkKr', 'hW45FMAE', 'VRFd8B7l'],
 '2ivUqLHP': 

In [15]:
train_pub['wlMo36J1']

{'abstract': "With the trend of globalization, enterprises require to extend their logistics scope from domestic to international, integrate logistics networks, and improve the agility and efficiency of global operations. This paper reviews the design and implementation of the industrial logistics hubs which belong to six manufacturing sectors in Taiwan over the past five years. Different industries have their own industry character, business strategy, and logistics models. Therefore, this paper organizes the executive problems and their corresponding solutions during project execution phases. Finally, this paper depicts the companies' executive experience and presents the future outlook for other companies and industry sectors' reference. © 2009 IEEE.",
 'authors': [{'name': 'Charles V. Trappey',
   'org': 'Dept. of Management Science'},
  {'name': 'Amy J.C. Trappey',
   'org': 'Dept. of industrial Engineering and Engineering Management'},
  {'name': 'Gilbert YP. LinC',
   'org': 'Dep

In [16]:
train_pub['dfcJQvnq']

{'abstract': 'In this paper, the effect of waste glass powder on properties and microstructure of ultrahigh performance cement based materials was investigated in an attempt to provide an additional option for glass recycling and to reduce part cost of the cementing materials. Compressive strength of pastes and mortars with different waste glass powder content was tested, and their microstructure properties were analysed by X-ray diffraction and scanning electron microscopy. Testing results show that the compressive strength rose initially with the increase in the waste glass amount, reached up to the peak at 20% replacement ratio and then decreased a little. The waste glass powder in ultrahigh performance cement based materials had exhibited pozzolanic reaction and filling effect, contributing substantially to strength increase. In addition, no alkali-silica reaction microcracking ring was found around the large glass powder particle due to the enough fineness of glass powder and the 

In [17]:
# tf-idf embediing
import re
import dill as pickle
tfidf_model = pickle.load(open('../my_data/tf_idf_model.pkl', mode="rb"))

with open(config.origin_data, mode='r', encoding='utf-8') as fo:
    paper_id_list = []
    paper_content_list = []
    for (line_index, line) in enumerate(fo):
        line = line.strip('\n').split('\t')
        paper_id, tittle, content, keywords = line[0], line[2], line[3], line[4]
        paper_id_list.append(paper_id)
        paper_content_list.append(' '.join([tittle, content, keywords]))

x_train = tfidf_model.transform(paper_content_list)

D:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [22]:
'microstructure' in tfidf_model.get_feature_names()

True

In [19]:
len(list(keyword_dict.keys()))

272010

In [20]:
len(tfidf_model.get_feature_names())

70577

In [24]:
train_author['']

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
